In [ ]:
import numpy as np

x = np.array([
    [1, 0], [0, 1], [0, -1], [-1, 0], [0, 2], [0, -2], [-2, 0]
])
y = np.array([-1, -1, -1, 1, 1, 1, 1])


In [ ]:
from sklearn import svm


def my_kernel(x1, x2):
    k = np.zeros((len(x1), len(x2)), dtype=np.float)
    for i in range(len(x1)):
        for j in range(len(x2)):
            x = x1[i]
            y = x2[j]
            k[i][j] = (1 + x.T.dot(y)) ** 2
    print(k.shape)
    return k


# svc = svm.SVC(kernel=my_kernel )
svc = svm.SVC(C=1e100, kernel='poly', degree=2, gamma=1, coef0=1)
svc.fit(x, y)

In [ ]:
a = np.sqrt(2)


def z(sv):
    return np.array([1, a * sv[0], a * sv[1], sv[0] ** 2, sv[1] ** 2, a * sv[0] * sv[1]])


def z2(sv):
    return np.array([1, sv[0], sv[1], sv[0] ** 2, sv[1] ** 2, sv[0] * sv[1]])

In [ ]:
print(np.sum(np.abs(svc.dual_coef_)))
print(svc.dual_coef_)
print(svc.support_)
print("support_vectors:", svc.support_vectors_)
w = np.zeros([6])
alpha = svc.dual_coef_.squeeze()
aIndex = 0
for index in svc.support_:
    print(x[index])
    print(z(x[index]))
    w += alpha[aIndex] * z(x[index])
    aIndex += 1
print("w:", w)
b = y[svc.support_[0]] - np.sum(w * z(x[svc.support_[0]]))
# b = svc.intercept_
print("b:", b)
w[0] += b
print("result:", (9 * w) * np.array([1, a, a, 1, 1, a]))
# print("result:", 9 * w)






In [ ]:
x_0 = x[:, 0]
x_1 = x[:, 1]
x_z = np.array([x_0, x_1, x_0 ** 2, x_1 ** 2, x_0 * x_1]).T
x_z = np.insert(x_z, 0, values=1, axis=1)
print(x_z)
svc = svm.SVC(kernel='linear')
svc.fit(x_z, y)
print(svc.coef_ * 9)







In [ ]:
from cvxopt import solvers, matrix
import numpy as np


def kernel(x1, x2):
    return (1 + np.dot(x1, x2)) ** 2


xs = [[1, 0], [0, 1], [0, -1], [-1, 0], [0, 2], [0, -2], [-2, 0]]
ys = np.array([-1, -1, -1, 1, 1, 1, 1])
Q = np.zeros((7, 7))
for i in range(7):
    for j in range(7):
        Q[i][j] = ys[i] * ys[j] * kernel(np.array(xs[i]), np.array(xs[j]))

p = -np.ones(7)
A = np.zeros((9, 7))
A[0] = ys
A[1] = -ys
for i in range(2, 9):
    A[i][i - 2] = -1
c = np.zeros(9)

P = matrix(Q)
q = matrix(p)
G = matrix(A)
h = matrix(c)

alphas = solvers.qp(P, q, G, h)
alphas = alphas['x']
print('max alpha:', np.max(alphas))
print('alpha sum:', np.sum(alphas))
print('min alpha:', np.min(alphas))
print('alphas:', alphas)


def kernelParameters(x):
    return np.array([x[0] * x[0], x[1] * x[1], 2 * x[0] * x[1], 2 * x[0], 2 * x[1], 1])


w = np.zeros(6)
for i in range(7):
    w += alphas[i] * ys[i] * kernelParameters(xs[i])

b = ys[1]
for i in range(7):
    b -= alphas[i] * ys[i] * kernel(xs[i], xs[1])

print('x1*x1:', w[0], 'x2*x2:', w[1], 'x1*x2:', w[2], 'x1:', w[3], 'x2:', w[4], '1:', w[5])
print('b:', b)



In [14]:
import os
import wget
import numpy as np

if not os.path.exists("features.train"):
    wget.download("http://www.amlbook.com/data/zip/features.train")
    wget.download("http://www.amlbook.com/data/zip/features.test")
train_data = np.loadtxt("features.train")
test_data = np.loadtxt("features.test")
print(train_data.shape, test_data.shape)

(7291, 3) (2007, 3)


In [10]:
x = train_data[:, 1:]
temp = train_data[:, 0]
y = temp.copy()
y[temp == 0] = 1
y[temp != 0] = -1

x_test = test_data[:, 1:-1]
temp = test_data[:, 0]
y_test = temp.copy()
y_test[temp == 0] = 1
y_test[temp != 0] = -1


NameError: name 'train_data' is not defined

In [11]:
svc = svm.SVC(C=0.01, kernel='linear')
svc.fit(x, y)
w = svc.coef_
b = svc.intercept_
print('w', w)
print('b', b)
print(np.linalg.norm(np.insert(w, 0, values=b, axis=1)))
print(np.linalg.norm(w))

NameError: name 'svm' is not defined

In [16]:
def experiment(kind, c, gamma=1, kernel='poly', train_data=train_data, test_data=test_data):
    x = train_data[:, 1:]
    temp = train_data[:, 0]
    y = temp.copy()
    y[temp == kind] = 1
    y[temp != kind] = -1
    x_test = test_data[:, 1:]
    temp = test_data[:, 0]
    y_test = temp.copy()
    y_test[temp == kind] = 1
    y_test[temp != kind] = -1

    svc = svm.SVC(C=c, kernel=kernel, gamma=gamma, degree=2, coef0=1)
    svc.fit(x, y)
    y_predict = svc.predict(x)
    error_in = np.sum(y != y_predict) / y.size
    sum_alpha = np.sum(np.abs(svc.dual_coef_))
    y_test_predict = svc.predict(x_test)
    error_out = np.sum(y_test != y_test_predict) / y_test.size
    # print("sv", svc.n_support_, "gamma", gamma)
    # print("kind", kind, "error_in", error_in, "sum_alpha", sum_alpha, "error_out", error_out, "C", c)
    return error_out


In [ ]:
for kind in [0, 2, 4, 6, 8]:
    experiment(kind, 0.01)

In [ ]:
for c in [0.001, 0.01, 0.1, 1, 10]:
    experiment(0, c, kernel='rbf', gamma=100)


In [ ]:
for exp in range(0, 5):
    gamma = 10 ** exp
    experiment(0, 0.1, gamma, 'rbf')


In [19]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn import svm
for exp in range(0, 5):
    e_val_sum = 0;
    gamma = 10 ** exp
    for _ in tqdm(range(100)):
        train_data_temp, val_data_temp = train_test_split(train_data, test_size= 1000/train_data.shape[0])
        e_val = experiment(kind=0, gamma=gamma, c=0.1, kernel='rbf', train_data=train_data_temp,
                           test_data=train_data_temp)
        e_val_sum += e_val
    print("gamma", gamma, "e_val", e_val_sum / 100)



  0%|          | 0/100 [00:00<?, ?it/s]

gamma 1 e_val 0.10608011444921317


  0%|          | 0/100 [00:00<?, ?it/s]

gamma 10 e_val 0.09831346367827054


  0%|          | 0/100 [00:00<?, ?it/s]

gamma 100 e_val 0.09609124145604828


  0%|          | 0/100 [00:00<?, ?it/s]

gamma 1000 e_val 0.16390875854395162


  3%|▎         | 3/100 [00:06<03:26,  2.13s/it]


KeyboardInterrupt: 

In [18]:
import sklearn.svm as svm
import numpy as np
import random



def getData(file):
    f = open(file,'r')
    lines = f.readlines()
    X = []
    Y = []
    for line in lines:
        data = line.strip().split()
        Y.append(int(float(data[0])))
        X.append([float(data[1]),float(data[2])])
    return np.array(X), np.array(Y)

def alignY(Y, target):
    newY = list(Y)
    for i in range(len(Y)):
        if Y[i] == target:
            newY[i] = 1
        else:
            newY[i] = -1
    return np.array(newY)

def computeError(X, Y, model):
    y_hat = np.array(model.predict(X))
    accNum = np.sum(y_hat == Y)
    return 1-accNum/len(Y)

if __name__ == '__main__':
    trainX, trainY = getData('features.train')
    testX, testY = getData('features.test')
    trainY15 = alignY(trainY,0)
    clf = svm.SVC(C=0.01,kernel='linear')
    clf.fit(trainX,trainY15)
    w = clf.coef_
    print('*********************************')
    print('p15:\nlength of w:',np.sqrt(np.sum(np.square(w))))

    E_in = 1
    cate = 0
    sumAlpha = 0
    for i in range(0,9,2):
        print(i)
        trainY16 = alignY(trainY,i)
        clf = svm.SVC(C=0.01,kernel='poly',degree=2,gamma='auto')
        clf.fit(trainX,trainY16)
        print(clf.dual_coef_)
        sumA = np.sum(np.fabs(clf.dual_coef_[0]))
        err = computeError(trainX,trainY16,clf)
        if err < E_in:
            E_in = err
            cate = i

        if sumA > sumAlpha:
            sumAlpha = sumA
    print('*********************************')
    print('p16:\nmin error rate cate is:',cate)
    print('*********************************')
    print('p17:\nmax alpha sum is:',sumAlpha)

    Cs = [0.001,0.01,0.1,1,10]
    trainY18 = alignY(trainY,0)
    testY18 = alignY(testY,0)
    for C in Cs:
        clf = svm.SVC(C=C,kernel='rbf',gamma=100)
        clf.fit(trainX,trainY18)

        print('C=',C)

        support = clf.support_
        coef = clf.dual_coef_[0]
        b = clf.intercept_[0]

        print('number of SV:',len(support))

        e_out = computeError(testX,testY18,clf)
        print('E_out:',e_out)

    gammas = [1,10,100,1000,10000]
    errorRate = 1
    g = 1
    for gamma in gammas:
        clf = svm.SVC(C=0.1,kernel='rbf',gamma=gamma)
        clf.fit(trainX,trainY18)

        e_out = computeError(testX,testY18,clf)
        if e_out < errorRate:
            errorRate = e_out
            g = gamma

    print('*********************************')
    print('p19:\nmin test error rate is:',errorRate, 'gamma:',g)

    gammas = [1,10,100,1000,10000]
    count = {1:0,10:0,100:0,1000:0,10000:0}
    indexs = list(range(len(trainY18)))

    for t in range(100):
        if t%10 == 9:
            print(t+1)
        errorRate = 1
        g = 1
        random.shuffle(indexs)
        for gamma in gammas:
            clf = svm.SVC(C=0.1,kernel='rbf',gamma=gamma)
            clf.fit(trainX[indexs[1000:]],trainY18[indexs[1000:]])

            e_out = computeError(trainX[indexs[:1000]],trainY18[indexs[:1000]],clf)
            if e_out < errorRate:
                errorRate = e_out
                g = gamma
        count[g] += 1

    print('*********************************')
    print('p20:\n most frequently gamma:',max(count.items(),key = lambda x:x[1]))



*********************************
p15:
length of w: 0.5713171494256942
0
[[-0.01 -0.01 -0.01 ...  0.01  0.01  0.01]]
2
[[-0.01 -0.01 -0.01 ...  0.01  0.01  0.01]]
4
[[-0.01 -0.01 -0.01 ...  0.01  0.01  0.01]]
6
[[-0.01 -0.01 -0.01 ...  0.01  0.01  0.01]]
8
[[-0.01 -0.01 -0.01 ...  0.01  0.01  0.01]]
*********************************
p16:
min error rate cate is: 8
*********************************
p17:
max alpha sum is: 23.88
C= 0.001
number of SV: 2398
E_out: 0.17887394120577982
C= 0.01
number of SV: 2520
E_out: 0.17887394120577982
C= 0.1
number of SV: 2285
E_out: 0.10513203786746383
C= 1
number of SV: 1783
E_out: 0.10363726955655206
C= 10
number of SV: 1675
E_out: 0.10463378176382665
*********************************
p19:
min test error rate is: 0.09915296462381662 gamma: 10
10
20
30
40
50
60
70
80
90
100
*********************************
p20:
 most frequently gamma: (10, 73)
